In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# constants
ROOT_PATH = 'C:/AgodaGit/passivbot' 
DOWNLOADED_CONFIG_PATH = ROOT_PATH + '/downloaded_configs/passivbot_v5.9.x'

In [5]:

import glob
import os

# list all json files in cfgs_bkup and its child folders
live_configs = glob.glob(ROOT_PATH + '/cfgs_bkup/**/*.json', recursive=True)
# normalize json_files by replacing all \\ with /
live_configs = [f.replace('\\', '/') for f in live_configs]


#group json_files by immediate parent folder
config_by_version = {}
for f in live_configs:
    folder = os.path.dirname(f).split('/')[-1]
    # folder looks like this : 5_9_0 or 5_9_3 or 5_9_4 etc. convert it to v5.9.0 or v.5.9.3 or v5.9.4
    folder = 'v' + folder.replace('_', '.')
    if folder not in config_by_version:
        config_by_version[folder] = {'live_configs': [] }
    config_by_version[folder]['live_configs'].append({
        'file': f,
        'coin': os.path.basename(f).replace('.json', '')
    })

config_by_version   


{'v5.9.0': {'live_configs': [{'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/ATOM.json',
    'coin': 'ATOM'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/CHZ.json', 'coin': 'CHZ'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/DOGE.json', 'coin': 'DOGE'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/DOT.json', 'coin': 'DOT'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/DYDX.json', 'coin': 'DYDX'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/EOS.json', 'coin': 'EOS'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/HBAR.json', 'coin': 'HBAR'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/LINK.json', 'coin': 'LINK'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/MANA.json', 'coin': 'MANA'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/MATIC.json',
    'coin': 'MATIC'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/OP.json', 'coin': 'OP'},
   {'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/SAND.json', 'coin': 'SAND'},
   {'file': 'C

In [6]:
# now find all config.json files in downloaded_configs/passivbot_v5.9.x folder
downloaded_configs = glob.glob(DOWNLOADED_CONFIG_PATH + '/**/config.json', recursive=True)
# normalize json_files by replacing all \\ with /
downloaded_configs = [f.replace('\\', '/') for f in downloaded_configs]


In [24]:
import re
'''
Result file looks like this:
    +------------------------------------------+
    |                 Summary                  |
    +-----------------------------+------------+
    | Metric                      | Value      |
    +-----------------------------+------------+
    | Exchange                    | bybit      |
    | Market type                 | futures    |
    | Symbol                      | ADAUSDT    |
    | Passivbot mode              | emas       |
    | No. days                    | 1100.0     |
    +-----------------------------+------------+
Write a parser function which takes in the path to result.txt file, reads it and then split the Metric and Value columns and put them in a dictionary and return it
'''
def parse_result_file(result_file):
    with open(result_file, 'r') as file:

        pattern = r'/v(\d+\.\d+(?:\.\w+)?)/'
        match = re.search(pattern, result_file)
        if match:
            version = match.group(1)
        else:
            version = None
            
        
        data = file.read().split('\n')
        d = {'version':version}
        current_state = None
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()
                # dont add if key is empty string
                if key == '':
                    continue

                if key == 'Long':
                    current_state = 'L'
                if key == 'Short':
                    current_state = 'S'
                
                # check if value can be converted to float and if so, convert it
                try:
                    value = value.replace('%', '').strip()
                    value = float(value)
                except:
                    pass
                   
                # if current_state is not None, then add it to key with a prefix of (L) or (S)
                if current_state is not None:
                    key = f'({current_state}){key}'
                d[key] = value
    return d

# test it
parse_result_file(r'C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/v5.9.5/recursive/july_2023/recursive_july_03_auenabled_0.2WE/RENUSDT_20230714162727_dfc69/RENUSDT/result.txt')

{'version': '5.9.5',
 'Metric': 'Value',
 'Exchange': 'bybit',
 'Market type': 'futures',
 'Symbol': 'RENUSDT',
 'Passivbot mode': 'recursive_grid',
 'ADG n subdivisions': 10.0,
 'No. days': 560.0,
 'Starting balance': 1000.0,
 '(L)Long': 'True',
 '(L)ADG per exposure': 0.0545,
 '(L)ADG weighted per exposure': 0.0796,
 '(L)Final balance': 1063.5,
 '(L)Final equity': 1063.5,
 '(L)Net PNL + fees': 63.5003,
 '(L)Net Total gain': 6.35,
 '(L)Average daily gain': 0.0109,
 '(L)Average daily gain weighted': 0.0159,
 '(L)Loss to profit ratio': 0.00014,
 '(L)Exposure ratios mean': 0.0084203,
 '(L)Price action distance mean': 0.0108677,
 '(L)Price action distance std': 0.00834529,
 '(L)Price action distance max': 0.0491622,
 '(L)Closest bankruptcy': 100.0,
 '(L)Lowest equity/balance ratio': 0.9737,
 '(L)Mean of 10 worst eq/bal ratios': 0.9921,
 '(L)Equity/balance ratio std': 0.0003235,
 '(L)No. fills': 17730.0,
 '(L)No. entries': 5623.0,
 '(L)No. closes': 12107.0,
 '(L)No. initial entries': 4623.

In [15]:
import json

_l_config = set()
# now iterate over config_by_version and find the downloaded config.json file for each live config
for version in config_by_version:
    for live_config in config_by_version[version]['live_configs']:
        coin = live_config['coin']+'USDT'
        file_content = json.load(open(live_config['file']))
        # find the list of matches in downloaded config.json files for this coin where version must be in the path and coin must be in the path
        matches = [f for f in downloaded_configs if version in f and coin in f]

        match_found = None
        #now find the exact match where the json.load of config.json is the exact same json as file_content
        for match in matches:
            if file_content == json.load(open(match)):
                match_found = match
                break
        if match_found is None:
            print(f'No match found for {coin} in {version}')
        else:
            _l_config.add(match_found)
            live_config['downloaded_config'] = match_found
            # find the backtest result file ( called result.txt ) for this downloaded config.json file. it is found in a subdirectory within the same folder as config.json
            #print(glob.glob(os.path.dirname(match_found)))
            result_file = glob.glob(os.path.dirname(match_found) + '/**/result.txt', recursive=True)
            live_config['result_file'] = result_file[0].replace('\\', '/') if len(result_file) > 0 else None
            live_config['result'] = parse_result_file(live_config['result_file']) if live_config['result_file'] is not None else None
            live_config['result']['version'] = version
            
config_by_version   
        

{'v5.9.0': {'live_configs': [{'file': 'C:/AgodaGit/passivbot/cfgs_bkup/5_9_0/ATOM.json',
    'coin': 'ATOM',
    'downloaded_config': 'C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/recursive/longshort/harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off/ATOMUSDT_20221222000537_d44da/config.json',
    'result_file': 'C:/AgodaGit/passivbot/downloaded_configs/passivbot_v5.9.x/configs/older_versions/v5.9.0/harmony/recursive/longshort/harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off/ATOMUSDT_20221222000537_d44da/ATOMUSDT/result.txt',
    'result': {'version': 'v5.9.0',
     'Metric': 'Value',
     'Exchange': 'binance',
     'Market type': 'futures',
     'Symbol': 'ATOMUSDT',
     'No. days': 1019.49,
     'Starting balance': 1500.0,
     '(L)Long': 'True',
     '(L)ADG realized per exposure': 0.271,
     '(L)Final balance': 2995.62,
     '(L)Final equity': 2995.63,
     '(L)Net PNL + fees': 1495.62,
     '(L)Total gain': 101.8,
 

In [25]:
# iterate over downloaded_configs, find result.txt file and parse it and add result to a pandas dataframe
import pandas as pd
result_analysis = pd.DataFrame()
for downloaded_config in downloaded_configs:     
    result_file = glob.glob(os.path.dirname(downloaded_config) + '/**/result.txt', recursive=True)
    result_file = result_file[0].replace('\\', '/') if len(result_file) > 0 else None
    if result_file is not None:
        result = parse_result_file(result_file)
        if result['version'] is None:
            print(f'No version found for {result_file}')
        result['file'] = downloaded_config
        if downloaded_config not in _l_config:
            result['live'] = False
        else:
            result['live'] = True
        # convert result to a pandas dataframe and append it to result_analysis dataframe
        result = pd.DataFrame(result, index=[0])
        result_analysis = pd.concat([result_analysis, result], ignore_index=True)

result_analysis

,version,Metric,Exchange,Market type,Symbol,No. days,Starting balance,(L)Long,(L)ADG realized per exposure,(L)Final balance,...,(L)ADG weighted per exposure,(L)Net Total gain,(L)Average daily gain weighted,(L)Mean of 10 worst eq/bal ratios,(L)Exposure ratios mean,(S)ADG per exposure,(S)ADG weighted per exposure,(S)Net Total gain,(S)Average daily gain weighted,(S)Mean of 10 worst eq/bal ratios
0,5.9.0,Value,binance,futures,ATOMUSDT,1046.000,1500.0,True,0.0954,4068.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.9.0,Value,binance,futures,CHZUSDT,705.708,1500.0,True,0.2780,10654.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.9.0,Value,binance,futures,DOGEUSDT,900.625,1500.0,True,0.4130,61566.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.9.0,Value,binance,futures,DOTUSDT,857.708,1500.0,True,0.2910,18083.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.9.0,Value,binance,futures,DYDXUSDT,473.854,1500.0,True,0.2640,5223.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,5.9.5,Value,bybit,futures,XRPUSDT,1200.000,1000.0,True,NaN,3104.15,...,0.1590,210.40,0.0476,0.9269,NaN,NaN,NaN,NaN,NaN,NaN
7724,5.9.5,Value,bybit,futures,XTZUSDT,1200.000,1000.0,True,NaN,2016.18,...,0.0801,101.60,0.0240,0.9142,NaN,NaN,NaN,NaN,NaN,NaN
7725,5.9.5,Value,bybit,futures,ZECUSDT,1200.000,1000.0,True,NaN,1489.54,...,0.0766,48.95,0.0230,0.9140,NaN,NaN,NaN,NaN,NaN,NaN
7726,5.9.5,Value,bybit,futures,ZILUSDT,1100.000,1000.0,True,NaN,2212.15,...,0.1530,121.20,0.0459,0.9580,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# list all columns in result_analysis, one column per line , wrapped in single quotes and separated by comma
print('\n'.join([f"'{c}'," for c in result_analysis.columns]))

'Metric',
'Exchange',
'Market type',
'Symbol',
'No. days',
'Starting balance',
'(L)Long',
'(L)ADG realized per exposure',
'(L)Final balance',
'(L)Final equity',
'(L)Net PNL + fees',
'(L)Total gain',
'(L)Average daily gain',
'(L)Loss to profit ratio',
'(L)Price action distance mean',
'(L)Price action distance std',
'(L)Price action distance max',
'(L)Closest bankruptcy',
'(L)Lowest equity/balance ratio',
'(L)Equity/balance ratio std',
'(L)No. fills',
'(L)No. entries',
'(L)No. closes',
'(L)No. initial entries',
'(L)No. reentries',
'(L)No. unstuck entries',
'(L)No. unstuck closes',
'(L)No. normal closes',
'(L)Average n fills per day',
'(L)Mean hours stuck',
'(L)Max hours stuck',
'(L)PNL sum',
'(L)Profit sum',
'(L)Loss sum',
'(L)Fee sum',
'(L)Biggest pos size',
'(L)Biggest pos cost',
'(L)Volume quote',
'file',
'live',
'(S)Short',
'(S)ADG realized per exposure',
'(S)Final balance',
'(S)Final equity',
'(S)Net PNL + fees',
'(S)Total gain',
'(S)Average daily gain',
'(S)Loss to profit ratio',
'

In [28]:
import requests

def get_minimum_qty(symbol):
    api_url = f"https://fapi.binance.com/fapi/v1/exchangeInfo"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        for symbol_info in data['symbols']:
            if symbol_info['symbol'] == symbol:
                return float(symbol_info['filters'][1]['stepSize'])
        return None
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

symbol = "BTCUSDT"  # Replace with the symbol you're interested in
minimum_qty = get_minimum_qty(symbol)

if minimum_qty is not None:
    print(f"Minimum quantity for {symbol}: {minimum_qty}")
else:
    print(f"Failed to get minimum quantity for {symbol}")


Minimum quantity for BTCUSDT: 0.001


In [26]:
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
# drop rows from result_analysis where symbol is not in allowed_symbols
result_analysis_filtered = result_analysis[result_analysis['Symbol'].isin(allowed_symbols)]

result_analysis_filtered = result_analysis_filtered[[
    'Symbol',
    'Exchange',
    'version',
    'No. days',
    'Passivbot mode',
    'live',
    
    '(L)Long',
    '(L)ADG realized per exposure',
    '(L)ADG weighted per exposure',
    '(L)Average daily gain',
    '(L)Average daily gain weighted',
    '(L)DG mean std ratio',
    '(L)ADG per exposure',
    '(L)Exposure ratios mean',
    '(L)Loss to profit ratio',
    '(L)Price action distance mean',
    '(L)Price action distance std',
    '(L)Price action distance max',
    '(L)Closest bankruptcy',
    '(L)Lowest equity/balance ratio',
    '(L)Mean of 10 worst eq/bal ratios',
    '(L)Equity/balance ratio std',
    '(L)No. initial entries',
    '(L)No. reentries',
    '(L)No. unstuck entries',
    '(L)No. unstuck/EMA entries',
    '(L)No. unstuck closes',
    '(L)No. unstuck/EMA closes',
    '(L)No. normal closes',
    '(L)Mean hours stuck',
    '(L)Max hours stuck',



    
    '(S)Short',
    '(S)ADG realized per exposure',
    '(S)ADG weighted per exposure',
    '(S)Average daily gain',
    '(S)Average daily gain weighted',
    '(S)DG mean std ratio',
    '(S)ADG per exposure',
    '(S)Loss to profit ratio',
    '(S)Price action distance mean',
    '(S)Price action distance std',
    '(S)Price action distance max',
    '(S)Closest bankruptcy',
    '(S)Lowest equity/balance ratio',
    '(S)Mean of 10 worst eq/bal ratios',
    '(S)Equity/balance ratio std',
    '(S)No. initial entries',
    '(S)No. reentries',
    '(S)No. unstuck entries',
    '(S)No. unstuck/EMA entries',
    '(S)No. unstuck closes',
    '(S)No. unstuck/EMA closes',
    '(S)No. normal closes',
    '(S)Mean hours stuck',
    '(S)Max hours stuck',

    'file',

]]

result_analysis_filtered

,Symbol,Exchange,version,No. days,Passivbot mode,live,(L)Long,(L)ADG realized per exposure,(L)ADG weighted per exposure,(L)Average daily gain,...,(S)No. initial entries,(S)No. reentries,(S)No. unstuck entries,(S)No. unstuck/EMA entries,(S)No. unstuck closes,(S)No. unstuck/EMA closes,(S)No. normal closes,(S)Mean hours stuck,(S)Max hours stuck,file
0,ATOMUSDT,binance,5.9.0,1046.000,NaN,False,True,0.0954,NaN,0.1290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
1,CHZUSDT,binance,5.9.0,705.708,NaN,False,True,0.2780,NaN,0.2660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
2,DOGEUSDT,binance,5.9.0,900.625,NaN,False,True,0.4130,NaN,0.4240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
3,DOTUSDT,binance,5.9.0,857.708,NaN,False,True,0.2910,NaN,0.3000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
5,EOSUSDT,binance,5.9.0,1084.670,NaN,False,True,0.1670,NaN,0.1730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7721,WAVESUSDT,bybit,5.9.5,1000.000,recursive_grid,False,True,NaN,0.1900,0.0666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
7723,XRPUSDT,bybit,5.9.5,1200.000,recursive_grid,False,True,NaN,0.1590,0.0920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
7724,XTZUSDT,bybit,5.9.5,1200.000,recursive_grid,False,True,NaN,0.0801,0.0584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...
7725,ZECUSDT,bybit,5.9.5,1200.000,recursive_grid,False,True,NaN,0.0766,0.0332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/downloaded_configs/passi...


In [27]:
# drop Metric column and move file column to the beginning
result_analysis_filtered.to_csv(ROOT_PATH+'/analysis_scripts/result_analysis.csv')


In [21]:
import glob
import json
import os
import pandas as pd


def contained(a, b):
    """ checks if dictionary a is fully contained in b """
    if not isinstance(a, dict):
        return a == b
    else:
        return all(contained(v, b.get(k)) for k, v in a.items())


big_data = pd.DataFrame()
## find current folder root
root = os.getcwd()

## get all list of json files in cfgs_live folder and put it in a list
live_configs = glob.glob('C:\\AgodaGit\\passivbot' + '/cfgs_live/*.json')
current_configs = {}
for file in live_configs:
    with open(file) as f:
        data = json.load(f)
        # extract filename from f
        currency = os.path.basename(file).replace('.json', '') + 'USDT'
        current_configs[currency] = data

live_configs = glob.iglob('passivbot_v5.8.0/configs' + '/**/config.json', recursive=True)
files = []
for file in live_configs:
    with open(file) as f:
        data = json.load(f)
        x = file.replace(root, '').replace('\\', '/')

        ## find the folder containing the config.json file
        config_folder = file.replace('\\', '/').replace('/config.json', '')
        ## find the result.txt file
        result_txt = glob.iglob(config_folder + '/**/result.txt', recursive=True)
        ## if result_txt list is not empty, get the first result
        result_txt = next(result_txt, None)

        if result_txt is not None:
            result_file = result_txt.replace('\\', '/')
            currency = result_txt.replace('\\', '/').replace('/result.txt', '').split('/')[-1]
            if currency in current_configs:
                current_config = current_configs[currency]
                ## compare each property current_config json object with data json object recursively and see if they are equal
                if contained(current_config, data):
                    print('Found config file for ' + currency)
                    files.append(result_file)

i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = os.path.abspath(filename.replace('result.txt', '../config.json'))
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        big_data = big_data.append(d, ignore_index=True)
big_data

Found config file for CHZUSDT
Found config file for MATICUSDT
Found config file for DOGEUSDT
Found config file for EOSUSDT
Found config file for LINKUSDT
Found config file for XRPUSDT
Found config file for ATOMUSDT
Found config file for HBARUSDT


C:\Users\skhan\AppData\Local\Temp\ipykernel_18552\639289630.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_18552\639289630.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_18552\639289630.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_18552\639289630.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds,short,tgs_%
0,passivbot_v5.8.0/configs/particle_swarm/neat/n...,CHZUSDT,True,16.84,0.0227,0.010568,0.972791,100.0,0.9507,3.097221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,passivbot_v5.8.0/configs/particle_swarm/recurs...,MATICUSDT,True,110.70,0.0980,0.009527,0.344123,100.0,0.9358,3.534721,0.0226,100.0,0.9546,4.047917,0.274453,0.015690,True,18.71
2,passivbot_v5.8.0/configs/particle_swarm/recurs...,DOGEUSDT,True,78.90,0.0653,0.009553,0.444083,100.0,0.9230,14.734708,0.0308,100.0,0.8412,9.754875,0.569193,0.038511,True,28.69
3,passivbot_v5.8.0/configs/particle_swarm/recurs...,EOSUSDT,True,44.94,0.0354,0.010325,0.384362,100.0,0.9305,9.236125,0.0175,100.0,0.9474,6.215958,0.303643,0.013407,True,19.93
4,passivbot_v5.8.0/configs/particle_swarm/recurs...,LINKUSDT,True,69.82,0.0505,0.010666,0.423959,100.0,0.9255,4.113887,0.0198,100.0,0.9792,5.515292,0.146795,0.011554,True,22.96
5,passivbot_v5.8.0/configs/particle_swarm/recurs...,XRPUSDT,True,78.78,0.0552,0.009951,0.323725,100.0,0.9387,9.106250,0.0179,100.0,0.9449,6.441667,0.313927,0.009566,True,21.38
6,passivbot_v5.8.0/configs/particle_swarm/recurs...,ATOMUSDT,True,43.89,0.0353,0.009951,0.250914,100.0,0.9497,3.324304,0.0231,100.0,0.9728,3.318054,0.182261,0.010759,True,27.06
7,passivbot_v5.8.0/configs/particle_swarm/recurs...,HBARUSDT,True,58.97,0.0750,0.009953,0.640497,100.0,0.9023,5.772208,0.0538,100.0,0.9515,4.806250,0.287523,0.012303,True,40.02


In [22]:
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
#allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
result_analysis = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
result_analysis.to_csv('5.8.csv')
result_analysis

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds,short,tgs_%,adgt_%
0,passivbot_v5.8.0/configs/particle_swarm/neat/n...,CHZUSDT,True,16.84,0.0227,0.010568,0.972791,100.0,0.9507,3.097221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,passivbot_v5.8.0/configs/particle_swarm/recurs...,MATICUSDT,True,110.70,0.0980,0.009527,0.344123,100.0,0.9358,3.534721,0.0226,100.0,0.9546,4.047917,0.274453,0.015690,True,18.71,0.1206
2,passivbot_v5.8.0/configs/particle_swarm/recurs...,DOGEUSDT,True,78.90,0.0653,0.009553,0.444083,100.0,0.9230,14.734708,0.0308,100.0,0.8412,9.754875,0.569193,0.038511,True,28.69,0.0961
3,passivbot_v5.8.0/configs/particle_swarm/recurs...,EOSUSDT,True,44.94,0.0354,0.010325,0.384362,100.0,0.9305,9.236125,0.0175,100.0,0.9474,6.215958,0.303643,0.013407,True,19.93,0.0529
4,passivbot_v5.8.0/configs/particle_swarm/recurs...,LINKUSDT,True,69.82,0.0505,0.010666,0.423959,100.0,0.9255,4.113887,0.0198,100.0,0.9792,5.515292,0.146795,0.011554,True,22.96,0.0703
5,passivbot_v5.8.0/configs/particle_swarm/recurs...,XRPUSDT,True,78.78,0.0552,0.009951,0.323725,100.0,0.9387,9.106250,0.0179,100.0,0.9449,6.441667,0.313927,0.009566,True,21.38,0.0731
6,passivbot_v5.8.0/configs/particle_swarm/recurs...,ATOMUSDT,True,43.89,0.0353,0.009951,0.250914,100.0,0.9497,3.324304,0.0231,100.0,0.9728,3.318054,0.182261,0.010759,True,27.06,0.0584
7,passivbot_v5.8.0/configs/particle_swarm/recurs...,HBARUSDT,True,58.97,0.0750,0.009953,0.640497,100.0,0.9023,5.772208,0.0538,100.0,0.9515,4.806250,0.287523,0.012303,True,40.02,0.1288


In [23]:
result_analysis.drop_duplicates(
    subset=['s',  'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = result_analysis[['file', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl', 'pastdl',
         'maxss', 'pamaxs','pastds']]
dx

,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,passivbot_v5.8.0/configs/particle_swarm/neat/n...,CHZUSDT,NaN,0.0227,NaN,100.0,NaN,0.9507,NaN,3.097221,0.972791,0.010568,NaN,NaN,NaN
1,passivbot_v5.8.0/configs/particle_swarm/recurs...,MATICUSDT,0.1206,0.0980,0.0226,100.0,100.0,0.9358,0.9546,3.534721,0.344123,0.009527,4.047917,0.274453,0.015690
2,passivbot_v5.8.0/configs/particle_swarm/recurs...,DOGEUSDT,0.0961,0.0653,0.0308,100.0,100.0,0.9230,0.8412,14.734708,0.444083,0.009553,9.754875,0.569193,0.038511
3,passivbot_v5.8.0/configs/particle_swarm/recurs...,EOSUSDT,0.0529,0.0354,0.0175,100.0,100.0,0.9305,0.9474,9.236125,0.384362,0.010325,6.215958,0.303643,0.013407
4,passivbot_v5.8.0/configs/particle_swarm/recurs...,LINKUSDT,0.0703,0.0505,0.0198,100.0,100.0,0.9255,0.9792,4.113887,0.423959,0.010666,5.515292,0.146795,0.011554
5,passivbot_v5.8.0/configs/particle_swarm/recurs...,XRPUSDT,0.0731,0.0552,0.0179,100.0,100.0,0.9387,0.9449,9.106250,0.323725,0.009951,6.441667,0.313927,0.009566
6,passivbot_v5.8.0/configs/particle_swarm/recurs...,ATOMUSDT,0.0584,0.0353,0.0231,100.0,100.0,0.9497,0.9728,3.324304,0.250914,0.009951,3.318054,0.182261,0.010759
7,passivbot_v5.8.0/configs/particle_swarm/recurs...,HBARUSDT,0.1288,0.0750,0.0538,100.0,100.0,0.9023,0.9515,5.772208,0.640497,0.009953,4.806250,0.287523,0.012303
